Using this source for what nutrients are necessary:
https://www.hsph.harvard.edu/nutritionsource/vitamins/

and this source for all data:
https://fdc.nal.usda.gov/download-datasets.html

In [63]:
import pandas as pd
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
path = os.getcwd()
data_path = path + '\\data'

print(data_path)

D:\Git_Repositories\min-max_nutrition\data


In [64]:
foods = pd.read_csv(data_path + '\\foundation_food.csv')
food_cols_to_drop = ['NDB_number', 'footnote']
foods.drop(food_cols_to_drop, axis=1, inplace=True)
foods.head()
print(len(foods))

236


In [65]:
desc = pd.read_csv(data_path + '\\food.csv')
desc_cols_to_drop = ['data_type', 'food_category_id', 'publication_date']
desc.drop(desc_cols_to_drop, axis=1, inplace=True)

desc.head()
print(len(desc))

54960


In [66]:
foods = pd.merge(foods, desc, on='fdc_id')


#foods.head()

#print(len(merged_df))

In [67]:
nutrient_match = pd.read_csv(data_path + '\\food_nutrient.csv')
#nm_cols_to_drop = ["id", 'data_points', 'derivation_id', 'min', 'max', 'median', 'footnote', 'min_year_acqured']
#nutrient_match.drop(nm_cols_to_drop, axis=1, inplace=True)
nutrient_match.head()
#len(nutrient_match)

C:\Users\chris\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured
0,2201847,319877,1051,56.30,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,2201845,319877,1002,1.28,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,2201846,319877,1004,19.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,2201844,319877,1007,1.98,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,2201852,319878,1091,188.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [68]:
nutrient_match = nutrient_match.sort_values(by='nutrient_id', ascending=True)
print(nutrient_match['nutrient_id'].unique())

[1002 1003 1004 1005 1007 1008 1009 1010 1011 1012 1013 1014 1024 1032
 1039 1041 1043 1044 1050 1051 1062 1063 1075 1076 1077 1079 1082 1084
 1085 1087 1089 1090 1091 1092 1093 1094 1095 1097 1098 1100 1101 1102
 1103 1105 1106 1107 1108 1109 1110 1111 1112 1113 1114 1116 1117 1118
 1119 1120 1121 1122 1123 1125 1126 1127 1128 1129 1130 1131 1137 1146
 1159 1160 1161 1162 1165 1166 1167 1170 1175 1176 1177 1178 1180 1183
 1184 1185 1188 1191 1192 1194 1195 1196 1197 1198 1199 1210 1211 1212
 1213 1214 1215 1216 1217 1218 1219 1220 1221 1222 1223 1224 1225 1226
 1227 1228 1232 1253 1257 1258 1259 1260 1261 1262 1263 1264 1265 1266
 1267 1268 1269 1270 1271 1272 1273 1276 1277 1278 1279 1280 1281 1284
 1285 1286 1287 1288 1289 1292 1293 1294 1296 1298 1299 1300 1301 1303
 1304 1305 1306 1311 1312 1313 1314 1315 1316 1317 1321 1323 1325 1329
 1330 1331 1333 1334 1335 1340 1341 1404 1405 1406 1409 1411 1414 2000
 2003 2004 2005 2006 2007 2008 2009 2010 2012 2013 2014 2015 2016 2017
 2018 

In [69]:
nutrient_names = pd.read_csv(data_path + '\\nutrient.csv')
#nutrient_names = nutrient_names.sort_values(by='name', ascending=True)

nn_cols_to_drop = ["nutrient_nbr", 'rank']
nutrient_names.drop(nn_cols_to_drop, axis=1, inplace=True)

nutrient_names

,id,name,unit_name
0,2047,Energy (Atwater General Factors),KCAL
1,2048,Energy (Atwater Specific Factors),KCAL
2,1001,Solids,G
3,1002,Nitrogen,G
4,1003,Protein,G
5,1004,Total lipid (fat),G
6,1005,"Carbohydrate, by difference",G
7,1006,"Fiber, crude (DO NOT USE - Archived)",G
8,1007,Ash,G
9,1008,Energy,KCAL


In [70]:

nutrient_match = pd.merge(nutrient_match, nutrient_names[['id', 'name', 'unit_name']], left_on='nutrient_id', right_on='id', how='left')
nutrient_match.drop(['id', 'nutrient_id'], axis=1, inplace=True)
#nutrient_match.head()

KeyError: "['id'] not found in axis"

In [71]:
nutrient_match['nutrient_name'] = nutrient_match['name'] + ' (' + nutrient_match['unit_name'] + ')'
nutrient_match.drop(['name','unit_name'], axis=1, inplace=True)
#nutrient_match.head()

In [72]:
#foods.head()

In [73]:
nutrient_match.head()
#print(nutrient_match['nutrient_name'].unique())

,id_x,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,id_y,nutrient_name
0,21110464,1753172,1002,0.03,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1002.0,Nitrogen (G)
1,2268098,334192,1002,2.98,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1002.0,Nitrogen (G)
2,8531743,749091,1002,7.28,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1002.0,Nitrogen (G)
3,2268180,334194,1002,3.04,42.0,1.0,2.56,3.36,3.03,NaN,2011.0,1002.0,Nitrogen (G)
4,2268228,334198,1002,0.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1002.0,Nitrogen (G)


In [74]:
piv = nutrient_match.pivot(columns='nutrient_name', index= 'fdc_id', values='amount')

#piv.head()

In [85]:
df1 = foods
df2 = nutrient_match

# Assume 'df1' has columns 'fdc_id' and 'description'
# Assume 'df2' has columns 'fdc_id', 'amount', and 'nutrient_name'

# Merge df1 and df2 based on 'fdc_id'
merged_df = pd.merge(df1, df2[['fdc_id', 'amount', 'nutrient_name']], on='fdc_id', how='right')

# Pivot the merged DataFrame to create columns for unique nutrient_names
pivoted_df = merged_df.pivot(index='fdc_id', columns='nutrient_name', values='amount')

# Merge the pivoted DataFrame back to df1 based on 'fdc_id'
final_df = pd.merge(df1, pivoted_df, on='fdc_id')


final_df.head()


,fdc_id,description,NaN,"Ergosta-5,7-dienol (MG)","Ergosta-7,22-dienol (MG)",10-Formyl folic acid (10HCOFA) (UG),25-hydroxycholecalciferol (UG),5-Formyltetrahydrofolic acid (5-HCOH4 (UG),5-methyl tetrahydrofolate (5-MTHF) (UG),Alanine (G),Arginine (G),Ash (G),Aspartic acid (G),Beta-glucan (G),Beta-sitostanol (MG),Beta-sitosterol (MG),Betaine (MG),Biotin (UG),"Boron, B (UG)",Brassicasterol (MG),"Calcium, Ca (MG)",Campestanol (MG),Campesterol (MG),"Carbohydrate, by difference (G)","Carbohydrate, by summation (G)","Carotene, alpha (UG)","Carotene, beta (UG)","Carotene, gamma (UG)",Cholesterol (MG),"Choline, free (MG)","Choline, from glycerophosphocholine (MG)","Choline, from phosphocholine (MG)","Choline, from phosphotidyl choline (MG)","Choline, from sphingomyelin (MG)","Choline, total (MG)",Citric acid (MG),"Cobalt, Co (UG)","Copper, Cu (MG)","Cryptoxanthin, alpha (UG)","Cryptoxanthin, beta (UG)",Cysteine (G),Cystine (G),Daidzein (MG),Daidzin (MG),Delta-5-avenasterol (MG),Delta-7-Stigmastenol (MG),Energy (Atwater General Factors) (KCAL),Energy (Atwater Specific Factors) (KCAL),Energy (KCAL),Energy (kJ),Ergosta-7-enol (MG),Ergosterol (MG),Ergothioneine (MG),"Fatty acids, total monounsaturated (G)","Fatty acids, total polyunsaturated (G)","Fatty acids, total saturated (G)","Fatty acids, total trans (G)","Fatty acids, total trans-dienoic (G)","Fatty acids, total trans-monoenoic (G)","Fatty acids, total trans-polyenoic (G)","Fiber, insoluble (G)","Fiber, soluble (G)","Fiber, total dietary (G)","Folate, total (UG)",Fructose (G),Galactose (G),Genistein (MG),Genistin (MG),Glucose (G),Glutamic acid (G),Glycine (G),Glycitin (MG),High Molecular Weight Dietary Fiber (HMWDF) (G),Histidine (G),Hydroxyproline (G),"Iodine, I (UG)","Iron, Fe (MG)",Isoleucine (G),Lactose (G),Leucine (G),Low Molecular Weight Dietary Fiber (LMWDF) (G),Lutein (UG),Lutein + zeaxanthin (UG),Lycopene (UG),Lysine (G),MUFA 12:1 (G),MUFA 14:1 c (G),MUFA 15:1 (G),MUFA 16:1 c (G),MUFA 17:1 (G),MUFA 17:1 c (G),MUFA 18:1 (G),MUFA 18:1 c (G),MUFA 20:1 (G),MUFA 20:1 c (G),MUFA 22:1 (G),MUFA 22:1 c (G),MUFA 22:1 n-11 (G),MUFA 22:1 n-9 (G),MUFA 24:1 c (G),"Magnesium, Mg (MG)",Malic acid (MG),Maltose (G),"Manganese, Mn (MG)",Methionine (G),"Molybdenum, Mo (UG)",Niacin (MG),"Nickel, Ni (UG)",Nitrogen (G),Oxalic acid (MG),PUFA 18:2 (G),PUFA 18:2 CLAs (G),PUFA 18:2 c (G),"PUFA 18:2 n-6 c,c (G)",PUFA 18:3 (G),PUFA 18:3 c (G),"PUFA 18:3 n-3 c,c,c (ALA) (G)","PUFA 18:3 n-6 c,c,c (G)",PUFA 18:3i (G),PUFA 18:4 (G),PUFA 20:2 c (G),"PUFA 20:2 n-6 c,c (G)",PUFA 20:3 (G),PUFA 20:3 c (G),PUFA 20:3 n-3 (G),PUFA 20:3 n-9 (G),PUFA 20:4 (G),PUFA 20:4 n-6 (G),PUFA 20:4c (G),PUFA 20:5 n-3 (EPA) (G),PUFA 20:5c (G),PUFA 22:2 (G),PUFA 22:3 (G),PUFA 22:4 (G),PUFA 22:5 c (G),PUFA 22:5 n-3 (DPA) (G),PUFA 22:6 c (G),PUFA 22:6 n-3 (DHA) (G),Pantothenic acid (MG),Phenylalanine (G),"Phosphorus, P (MG)",Phytoene (UG),Phytofluene (UG),"Phytosterols, other (MG)","Potassium, K (MG)",Proline (G),Protein (G),Pyruvic acid (MG),Quinic acid (MG),Raffinose (G),Retinol (UG),Riboflavin (MG),SFA 10:0 (G),SFA 11:0 (G),SFA 12:0 (G),SFA 14:0 (G),SFA 15:0 (G),SFA 16:0 (G),SFA 17:0 (G),SFA 18:0 (G),SFA 20:0 (G),SFA 21:0 (G),SFA 22:0 (G),SFA 23:0 (G),SFA 24:0 (G),SFA 4:0 (G),SFA 5:0 (G),SFA 6:0 (G),SFA 7:0 (G),SFA 8:0 (G),SFA 9:0 (G),"Selenium, Se (UG)",Serine (G),"Sodium, Na (MG)",Specific Gravity (SP_GR),Stachyose (G),Starch (G),Stigmastadiene (MG),Stigmasterol (MG),Sucrose (G),"Sugars, Total (G)","Sugars, total including NLEA (G)","Sulfur, S (MG)",TFA 14:1 t (G),TFA 16:1 t (G),TFA 18:1 t (G),TFA 18:2 t (G),TFA 18:2 t not further defined (G),TFA 18:3 t (G),TFA 20:1 t (G),TFA 22:1 t (G),Thiamin (MG),Threonine (G),"Tocopherol, beta (MG)","Tocopherol, delta (MG)","Tocopherol, gamma (MG)","Tocotrienol, alpha (MG)","Tocotrienol, beta (MG)","Tocotrienol, delta (MG)","Tocotrienol, gamma (MG)",Total dietary fiber (AOAC 2011.25) (G),Total fat (NLEA) (G),Total lipid (fat) (G),Tryptophan (G),Tyrosine (G),Valine (G),Verbascose (G),"Vit

In [87]:
# Assume 'df' is your DataFrame

# Count the number of NaN values in each column
nan_counts = final_df.isna().sum()

# Sort the columns based on the NaN counts
sorted_columns = nan_counts.sort_values().index.tolist()

# Print the sorted columns with the fewest NaN values to the most
print(sorted_columns)


['fdc_id', 'description', 'Water (G)', 'Total lipid (fat) (G)', 'Protein (G)', 'Zinc, Zn (MG)', 'Iron, Fe (MG)', 'Magnesium, Mg (MG)', 'Manganese, Mn (MG)', 'Calcium, Ca (MG)', 'Copper, Cu (MG)', 'Phosphorus, P (MG)', 'Potassium, K (MG)', 'Ash (G)', 'Carbohydrate, by difference (G)', 'Energy (Atwater General Factors) (KCAL)', 'Sodium, Na (MG)', 'Nitrogen (G)', 'Energy (Atwater Specific Factors) (KCAL)', 'Vitamin B-6 (MG)', 'Niacin (MG)', 'Thiamin (MG)', 'Riboflavin (MG)', 'Fiber, total dietary (G)', 'Selenium, Se (UG)', 'Energy (KCAL)', 'Energy (kJ)', 'Fructose (G)', 'Glucose (G)', 'Lactose (G)', 'Maltose (G)', 'Sucrose (G)', 'Sugars, Total (G)', 'Folate, total (UG)', 'Galactose (G)', 'Fatty acids, total saturated (G)', 'Fatty acids, total monounsaturated (G)', 'SFA 18:0 (G)', 'SFA 16:0 (G)', 'PUFA 18:2 n-6 c,c (G)', 'SFA 14:0 (G)', 'MUFA 18:1 c (G)', 'Biotin (UG)', 'SFA 12:0 (G)', 'Fatty acids, total polyunsaturated (G)', 'Molybdenum, Mo (UG)', 'MUFA 16:1 c (G)', 'MUFA 20:1 c (G)', 'S

In [89]:
print(nan_counts.sort_values(ascending=True))

fdc_id                                               0
description                                          0
Water (G)                                            8
Total lipid (fat) (G)                                9
Protein (G)                                         11
Zinc, Zn (MG)                                       12
Iron, Fe (MG)                                       12
Magnesium, Mg (MG)                                  12
Manganese, Mn (MG)                                  12
Calcium, Ca (MG)                                    12
Copper, Cu (MG)                                     12
Phosphorus, P (MG)                                  12
Potassium, K (MG)                                   12
Ash (G)                                             27
Carbohydrate, by difference (G)                     28
Energy (Atwater General Factors) (KCAL)             28
Sodium, Na (MG)                                     30
Nitrogen (G)                                        37
Energy (At

In [96]:
# Assume 'df' is your DataFrame

# Find the row(s) where the value in the "Ash (G)" column is NaN
rows_with_nan = final_df[final_df["Energy (KCAL)"].isna()]

# Print the row(s) with NaN value in the "Ash (G)" column
rows_with_nan.head()


,fdc_id,description,NaN,"Ergosta-5,7-dienol (MG)","Ergosta-7,22-dienol (MG)",10-Formyl folic acid (10HCOFA) (UG),25-hydroxycholecalciferol (UG),5-Formyltetrahydrofolic acid (5-HCOH4 (UG),5-methyl tetrahydrofolate (5-MTHF) (UG),Alanine (G),Arginine (G),Ash (G),Aspartic acid (G),Beta-glucan (G),Beta-sitostanol (MG),Beta-sitosterol (MG),Betaine (MG),Biotin (UG),"Boron, B (UG)",Brassicasterol (MG),"Calcium, Ca (MG)",Campestanol (MG),Campesterol (MG),"Carbohydrate, by difference (G)","Carbohydrate, by summation (G)","Carotene, alpha (UG)","Carotene, beta (UG)","Carotene, gamma (UG)",Cholesterol (MG),"Choline, free (MG)","Choline, from glycerophosphocholine (MG)","Choline, from phosphocholine (MG)","Choline, from phosphotidyl choline (MG)","Choline, from sphingomyelin (MG)","Choline, total (MG)",Citric acid (MG),"Cobalt, Co (UG)","Copper, Cu (MG)","Cryptoxanthin, alpha (UG)","Cryptoxanthin, beta (UG)",Cysteine (G),Cystine (G),Daidzein (MG),Daidzin (MG),Delta-5-avenasterol (MG),Delta-7-Stigmastenol (MG),Energy (Atwater General Factors) (KCAL),Energy (Atwater Specific Factors) (KCAL),Energy (KCAL),Energy (kJ),Ergosta-7-enol (MG),Ergosterol (MG),Ergothioneine (MG),"Fatty acids, total monounsaturated (G)","Fatty acids, total polyunsaturated (G)","Fatty acids, total saturated (G)","Fatty acids, total trans (G)","Fatty acids, total trans-dienoic (G)","Fatty acids, total trans-monoenoic (G)","Fatty acids, total trans-polyenoic (G)","Fiber, insoluble (G)","Fiber, soluble (G)","Fiber, total dietary (G)","Folate, total (UG)",Fructose (G),Galactose (G),Genistein (MG),Genistin (MG),Glucose (G),Glutamic acid (G),Glycine (G),Glycitin (MG),High Molecular Weight Dietary Fiber (HMWDF) (G),Histidine (G),Hydroxyproline (G),"Iodine, I (UG)","Iron, Fe (MG)",Isoleucine (G),Lactose (G),Leucine (G),Low Molecular Weight Dietary Fiber (LMWDF) (G),Lutein (UG),Lutein + zeaxanthin (UG),Lycopene (UG),Lysine (G),MUFA 12:1 (G),MUFA 14:1 c (G),MUFA 15:1 (G),MUFA 16:1 c (G),MUFA 17:1 (G),MUFA 17:1 c (G),MUFA 18:1 (G),MUFA 18:1 c (G),MUFA 20:1 (G),MUFA 20:1 c (G),MUFA 22:1 (G),MUFA 22:1 c (G),MUFA 22:1 n-11 (G),MUFA 22:1 n-9 (G),MUFA 24:1 c (G),"Magnesium, Mg (MG)",Malic acid (MG),Maltose (G),"Manganese, Mn (MG)",Methionine (G),"Molybdenum, Mo (UG)",Niacin (MG),"Nickel, Ni (UG)",Nitrogen (G),Oxalic acid (MG),PUFA 18:2 (G),PUFA 18:2 CLAs (G),PUFA 18:2 c (G),"PUFA 18:2 n-6 c,c (G)",PUFA 18:3 (G),PUFA 18:3 c (G),"PUFA 18:3 n-3 c,c,c (ALA) (G)","PUFA 18:3 n-6 c,c,c (G)",PUFA 18:3i (G),PUFA 18:4 (G),PUFA 20:2 c (G),"PUFA 20:2 n-6 c,c (G)",PUFA 20:3 (G),PUFA 20:3 c (G),PUFA 20:3 n-3 (G),PUFA 20:3 n-9 (G),PUFA 20:4 (G),PUFA 20:4 n-6 (G),PUFA 20:4c (G),PUFA 20:5 n-3 (EPA) (G),PUFA 20:5c (G),PUFA 22:2 (G),PUFA 22:3 (G),PUFA 22:4 (G),PUFA 22:5 c (G),PUFA 22:5 n-3 (DPA) (G),PUFA 22:6 c (G),PUFA 22:6 n-3 (DHA) (G),Pantothenic acid (MG),Phenylalanine (G),"Phosphorus, P (MG)",Phytoene (UG),Phytofluene (UG),"Phytosterols, other (MG)","Potassium, K (MG)",Proline (G),Protein (G),Pyruvic acid (MG),Quinic acid (MG),Raffinose (G),Retinol (UG),Riboflavin (MG),SFA 10:0 (G),SFA 11:0 (G),SFA 12:0 (G),SFA 14:0 (G),SFA 15:0 (G),SFA 16:0 (G),SFA 17:0 (G),SFA 18:0 (G),SFA 20:0 (G),SFA 21:0 (G),SFA 22:0 (G),SFA 23:0 (G),SFA 24:0 (G),SFA 4:0 (G),SFA 5:0 (G),SFA 6:0 (G),SFA 7:0 (G),SFA 8:0 (G),SFA 9:0 (G),"Selenium, Se (UG)",Serine (G),"Sodium, Na (MG)",Specific Gravity (SP_GR),Stachyose (G),Starch (G),Stigmastadiene (MG),Stigmasterol (MG),Sucrose (G),"Sugars, Total (G)","Sugars, total including NLEA (G)","Sulfur, S (MG)",TFA 14:1 t (G),TFA 16:1 t (G),TFA 18:1 t (G),TFA 18:2 t (G),TFA 18:2 t not further defined (G),TFA 18:3 t (G),TFA 20:1 t (G),TFA 22:1 t (G),Thiamin (MG),Threonine (G),"Tocopherol, beta (MG)","Tocopherol, delta (MG)","Tocopherol, gamma (MG)","Tocotrienol, alpha (MG)","Tocotrienol, beta (MG)","Tocotrienol, delta (MG)","Tocotrienol, gamma (MG)",Total dietary fiber (AOAC 2011.25) (G),Total fat (NLEA) (G),Total lipid (fat) (G),Tryptophan (G),Tyrosine (G),Valine (G),Verbascose (G),"Vit

In [78]:
#df1.head()

In [79]:
#df2.head()

In [80]:
df3 = df2.pivot(index='fdc_id', columns='nutrient_name', values='amount')
#df3.head()

In [81]:
#print(df1.nunique())

In [82]:
final_df.head()

,fdc_id,description,NaN,"Ergosta-5,7-dienol (MG)","Ergosta-7,22-dienol (MG)",10-Formyl folic acid (10HCOFA) (UG),25-hydroxycholecalciferol (UG),5-Formyltetrahydrofolic acid (5-HCOH4 (UG),5-methyl tetrahydrofolate (5-MTHF) (UG),Alanine (G),Arginine (G),Ash (G),Aspartic acid (G),Beta-glucan (G),Beta-sitostanol (MG),Beta-sitosterol (MG),Betaine (MG),Biotin (UG),"Boron, B (UG)",Brassicasterol (MG),"Calcium, Ca (MG)",Campestanol (MG),Campesterol (MG),"Carbohydrate, by difference (G)","Carbohydrate, by summation (G)","Carotene, alpha (UG)","Carotene, beta (UG)","Carotene, gamma (UG)",Cholesterol (MG),"Choline, free (MG)","Choline, from glycerophosphocholine (MG)","Choline, from phosphocholine (MG)","Choline, from phosphotidyl choline (MG)","Choline, from sphingomyelin (MG)","Choline, total (MG)",Citric acid (MG),"Cobalt, Co (UG)","Copper, Cu (MG)","Cryptoxanthin, alpha (UG)","Cryptoxanthin, beta (UG)",Cysteine (G),Cystine (G),Daidzein (MG),Daidzin (MG),Delta-5-avenasterol (MG),Delta-7-Stigmastenol (MG),Energy (Atwater General Factors) (KCAL),Energy (Atwater Specific Factors) (KCAL),Energy (KCAL),Energy (kJ),Ergosta-7-enol (MG),Ergosterol (MG),Ergothioneine (MG),"Fatty acids, total monounsaturated (G)","Fatty acids, total polyunsaturated (G)","Fatty acids, total saturated (G)","Fatty acids, total trans (G)","Fatty acids, total trans-dienoic (G)","Fatty acids, total trans-monoenoic (G)","Fatty acids, total trans-polyenoic (G)","Fiber, insoluble (G)","Fiber, soluble (G)","Fiber, total dietary (G)","Folate, total (UG)",Fructose (G),Galactose (G),Genistein (MG),Genistin (MG),Glucose (G),Glutamic acid (G),Glycine (G),Glycitin (MG),High Molecular Weight Dietary Fiber (HMWDF) (G),Histidine (G),Hydroxyproline (G),"Iodine, I (UG)","Iron, Fe (MG)",Isoleucine (G),Lactose (G),Leucine (G),Low Molecular Weight Dietary Fiber (LMWDF) (G),Lutein (UG),Lutein + zeaxanthin (UG),Lycopene (UG),Lysine (G),MUFA 12:1 (G),MUFA 14:1 c (G),MUFA 15:1 (G),MUFA 16:1 c (G),MUFA 17:1 (G),MUFA 17:1 c (G),MUFA 18:1 (G),MUFA 18:1 c (G),MUFA 20:1 (G),MUFA 20:1 c (G),MUFA 22:1 (G),MUFA 22:1 c (G),MUFA 22:1 n-11 (G),MUFA 22:1 n-9 (G),MUFA 24:1 c (G),"Magnesium, Mg (MG)",Malic acid (MG),Maltose (G),"Manganese, Mn (MG)",Methionine (G),"Molybdenum, Mo (UG)",Niacin (MG),"Nickel, Ni (UG)",Nitrogen (G),Oxalic acid (MG),PUFA 18:2 (G),PUFA 18:2 CLAs (G),PUFA 18:2 c (G),"PUFA 18:2 n-6 c,c (G)",PUFA 18:3 (G),PUFA 18:3 c (G),"PUFA 18:3 n-3 c,c,c (ALA) (G)","PUFA 18:3 n-6 c,c,c (G)",PUFA 18:3i (G),PUFA 18:4 (G),PUFA 20:2 c (G),"PUFA 20:2 n-6 c,c (G)",PUFA 20:3 (G),PUFA 20:3 c (G),PUFA 20:3 n-3 (G),PUFA 20:3 n-9 (G),PUFA 20:4 (G),PUFA 20:4 n-6 (G),PUFA 20:4c (G),PUFA 20:5 n-3 (EPA) (G),PUFA 20:5c (G),PUFA 22:2 (G),PUFA 22:3 (G),PUFA 22:4 (G),PUFA 22:5 c (G),PUFA 22:5 n-3 (DPA) (G),PUFA 22:6 c (G),PUFA 22:6 n-3 (DHA) (G),Pantothenic acid (MG),Phenylalanine (G),"Phosphorus, P (MG)",Phytoene (UG),Phytofluene (UG),"Phytosterols, other (MG)","Potassium, K (MG)",Proline (G),Protein (G),Pyruvic acid (MG),Quinic acid (MG),Raffinose (G),Retinol (UG),Riboflavin (MG),SFA 10:0 (G),SFA 11:0 (G),SFA 12:0 (G),SFA 14:0 (G),SFA 15:0 (G),SFA 16:0 (G),SFA 17:0 (G),SFA 18:0 (G),SFA 20:0 (G),SFA 21:0 (G),SFA 22:0 (G),SFA 23:0 (G),SFA 24:0 (G),SFA 4:0 (G),SFA 5:0 (G),SFA 6:0 (G),SFA 7:0 (G),SFA 8:0 (G),SFA 9:0 (G),"Selenium, Se (UG)",Serine (G),"Sodium, Na (MG)",Specific Gravity (SP_GR),Stachyose (G),Starch (G),Stigmastadiene (MG),Stigmasterol (MG),Sucrose (G),"Sugars, Total (G)","Sugars, total including NLEA (G)","Sulfur, S (MG)",TFA 14:1 t (G),TFA 16:1 t (G),TFA 18:1 t (G),TFA 18:2 t (G),TFA 18:2 t not further defined (G),TFA 18:3 t (G),TFA 20:1 t (G),TFA 22:1 t (G),Thiamin (MG),Threonine (G),"Tocopherol, beta (MG)","Tocopherol, delta (MG)","Tocopherol, gamma (MG)","Tocotrienol, alpha (MG)","Tocotrienol, beta (MG)","Tocotrienol, delta (MG)","Tocotrienol, gamma (MG)",Total dietary fiber (AOAC 2011.25) (G),Total fat (NLEA) (G),Total lipid (fat) (G),Tryptophan (G),Tyrosine (G),Valine (G),Verbascose (G),"Vit

In [83]:
nutrients_dict = {
    'Biotin (UG)' : 30,
    'Calcium, Ca (MG)' : 1000,
    'Chromium, Cr (UG)' : 35,
    'Choline, total (MG)' : 550,
    'Copper, Cu (MG)' : 0.9,
    'Folate, total (UG)' : 400,
    'Iodine, I (UG)' : 150,
    'Iron, Fe (MG)' : 8,
    'Magnesium, Mg (MG)' : 400,
    'Manganese, Mn (MG)' : 2.3,
    'Molybdenum, Mo (UG)' : 45,
    'Niacin (MG)' : 16,
    'Nickel, Ni (UG)' : 35,
    'Pantothenic acid (MG)' : 5,
    'Phosphorus, P (MG)' : 700,
    'Potassium, K (MG)' : 3400,
    'Riboflavin (MG)' : 1.3,
    'Selenium, Se (UG)' : 55,
    'Sodium, Na (MG)' : 1500,
    'Thiamin (MG)' : 1.2,
    'Vitamin A, RAE (UG)' : 900,
    'Vitamin B-12 (UG)' : 2.4,
    'Vitamin B-6 (MG)' : 1.3,
    'Vitamin C, total ascorbic acid (MG)' : 90,
    'Vitamin D (D2 + D3) (UG)' : 15,
    'Vitamin E (alpha-tocopherol) (MG)' : 15,
    'Vitamin K (phylloquinone) (UG)' : 120,
    'Zinc, Zn (MG)' : 11,
}

keep_columns = list(nutrients_dict.keys())
print(keep_columns[0])

Biotin (UG)


In [84]:
columns_to_drop = [col for col in final_df.columns if col not in keep_columns]
columns_to_drop.append('description')
final_df.drop(columns_to_drop, axis=1, inplace=True)
final_df.fillna(0, inplace=True)
final_df.head()

,Biotin (UG),"Calcium, Ca (MG)","Choline, total (MG)","Copper, Cu (MG)","Folate, total (UG)","Iodine, I (UG)","Iron, Fe (MG)","Magnesium, Mg (MG)","Manganese, Mn (MG)","Molybdenum, Mo (UG)",Niacin (MG),"Nickel, Ni (UG)",Pantothenic acid (MG),"Phosphorus, P (MG)","Potassium, K (MG)",Riboflavin (MG),"Selenium, Se (UG)","Sodium, Na (MG)",Thiamin (MG),"Vitamin A, RAE (UG)",Vitamin B-12 (UG),Vitamin B-6 (MG),"Vitamin C, total ascorbic acid (MG)",Vitamin D (D2 + D3) (UG),Vitamin E (alpha-tocopherol) (MG),Vitamin K (phylloquinone) (UG),"Zinc, Zn (MG)"
0,0.0,41.0,46.6,0.348,36.0,0.0,2.41,71.1,1.060,0.0,0.948,0.0,0.318,166.0,289.0,0.115,16.2,438.0,0.150,1.0,0.00,0.143,0.0,0.0,1.74,17.2,1.38
1,0.0,11.0,9.8,0.058,10.0,0.0,0.33,11.9,0.121,0.0,0.805,0.0,0.000,28.0,260.0,0.065,0.0,6.0,0.075,0.0,0.00,0.060,27.2,0.0,0.98,4.2,0.20
2,0.0,36.0,0.0,0.041,0.0,0.0,0.78,12.7,0.176,0.0,0.000,0.0,0.000,23.0,97.0,0.000,0.0,282.0,0.000,0.0,0.00,0.000,0.0,0.0,0.00,0.0,0.19
3,0.0,15.0,0.0,0.046,0.0,0.0,1.14,11.5,0.031,0.0,2.250,0.0,0.263,128.0,343.0,0.154,0.0,872.0,0.033,3.0,0.97,0.130,0.0,0.0,0.51,0.0,2.06
4,0.0,273.0,60.8,0.870,35.0,0.0,3.17,258.0,2.020,0.0,3.100,0.0,0.237,456.0,684.0,1.570,0.0,256.0,0.079,2.0,0.00,0.075,0.0,0.0,19.00,0.0,2.80
